In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import zipfile

In [2]:
covid = pd.read_csv("covid19_cleaned.zip", low_memory = False)
covid[:3]

,Unnamed: 0,Unnamed: 0.1,ID,age,sex,city,province,country,latitude,longitude,...,location,admin3,admin2,admin1,country_new,admin_id,data_moderator_initials,travel_history_binary,hospitalized,deceased
0,0,0,000-1-1,NaN,male,Shek Lei,Hong Kong,China,22.365019,114.133808,...,Shek Lei,NaN,NaN,Hong Kong,China,8051.0,NaN,NaN,1,0
1,1,1,000-1-10,78,male,Vo Euganeo,Veneto,Italy,45.297748,11.658382,...,Vo' Euganeo,NaN,NaN,Veneto,Italy,8978.0,NaN,NaN,0,1
2,2,2,000-1-100,61,female,NaN,NaN,Singapore,1.353460,103.815100,...,NaN,NaN,NaN,NaN,Singapore,201.0,NaN,NaN,0,0


In [3]:
dropcols = ["admin_id", "notes_for_discussion", "source",
            "geo_resolution", "lives_in_Wuhan", "additional_information",
            "sequence_available", "data_moderator_initials", "location",
           "country_new", "Unnamed: 0", "Unnamed: 0.1"]

covid = covid.drop(covid[dropcols], axis = 1)
print(covid.shape)

(2310111, 25)


In [4]:
#cleaning of the age column

age = np.array(covid.age)
sym = ["weeks", "+", "-", " "]
for x,y in enumerate(age):
    if type(y) is float:
        age[x] = y
    elif sym[0] in y:
        age[x] = 0
    elif sym[1] in y:
        age[x] = y.replace('+','')
        age[x] = int(round(float(age[x])))
    elif sym[2] in y:
        if len(y) >= 4:
            mat = y.split("-")
            if "Sep" in mat:
                age[x] = 0
            else:
                age[x] = (int(mat[0]) + int(mat[1]))/2
        else:
            age[x] = y.replace('-','')
            age[x] = int(round(float(age[x])))
    elif sym[3] in y:
        num, month = y.split(" ")
        num = int(num)
        if num < 6:
            age[x] = 0
        elif num >= 6 and num < 18:
            age[x] = 1
        else:
            age[x] = 2
for x,y in enumerate(age):
    age[x] = float(y)
covid["age"] = age
covid["age"].fillna(covid["age"].mean(), inplace = True)
covid['age'] = covid['age'].astype(float)

In [5]:
#converts bool to binary.

covid.chronic_disease_binary = [1 if x == True else 0 for x in covid.chronic_disease_binary]

In [6]:
#drop missing country values. conveniently drops missing lat and long vals.

covid = covid.dropna(subset = ["country"])

In [7]:
#nan becomes a zero, if they traveled it becomes a one

covid.travel_history_binary = [1 if x == True else 0 for x in covid["travel_history_binary"]]

In [8]:
# craft a boolean array and for values False (NaN), cut in half and assign half female/male??

sex_e = np.array(["male", "female"])
sex = np.array(covid.sex)

sex_is_bool = [True if x == "male" or x == "female" else False for x in covid["sex"]]
for x, y in enumerate(sex_is_bool):
    if y == False:
        sex[x] = np.random.choice(sex_e)
covid.sex = sex
covid.sex.value_counts()

female    1158147
male      1151849
Name: sex, dtype: int64

In [9]:
covid.to_csv("covid19cleaned_sexandothers.zip", compression = "zip")